In [2]:
#Mount gdrive to this notebook
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# #adding directory of the data we will be using

import os
# os.chdir('/content/drive/MyDrive/ColabNotebooks/AML_project/dataset/Train/Closed_Eyes')

In [4]:
import tensorflow as tf
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import InceptionV3

In [5]:
def eye_detection_train(eye_train,img_cluster):
  img_gen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2, zoom_range=0.2,width_shift_range=0.2, height_shift_range=0.2, validation_split=0.2) #generation of images in multiple angles
  eye_trd = img_gen.flow_from_directory(eye_train, target_size=(80,80),batch_size=img_cluster,class_mode='categorical',subset='training' )
  eye_vd = img_gen.flow_from_directory(eye_train, target_size=(80,80),batch_size=img_cluster,class_mode='categorical', subset='validation')
  return eye_trd,eye_vd


In [6]:
def yawn_detection_train(yawn_train,img_cluster):
  img_gen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2, zoom_range=0.2,width_shift_range=0.2, height_shift_range=0.2, validation_split=0.2) #generation of images in multiple angles
  yawn_trd = img_gen.flow_from_directory(yawn_train, target_size=(80,80),batch_size=img_cluster,class_mode='categorical',subset='training' )
  yawn_vd = img_gen.flow_from_directory(yawn_train, target_size=(80,80),batch_size=img_cluster,class_mode='categorical', subset='validation')
  return yawn_trd,yawn_vd


In [7]:
def eye_detection_test(eye_test,img_cluster):
  img_gen = ImageDataGenerator(rescale=1./255)
  eye_ted = img_gen.flow_from_directory(eye_test, target_size=(80,80),batch_size=img_cluster,class_mode='categorical')
  return eye_ted

In [8]:
def yawn_detection_test(yawn_test,img_cluster):
  img_gen = ImageDataGenerator(rescale=1./255)
  yawn_ted = img_gen.flow_from_directory(yawn_test, target_size=(80,80),batch_size=img_cluster,class_mode='categorical')
  return yawn_ted

In [9]:
eye_trd,eye_vd = eye_detection_train('/content/drive/MyDrive/ColabNotebooks/AML_project/dataset/Train/' , 12)

yawn_trd,yawn_vd = yawn_detection_train('/content/drive/MyDrive/ColabNotebooks/AML_project/dataset/Train_y/' , 12) 

eye_ted = eye_detection_test('/content/drive/MyDrive/ColabNotebooks/AML_project/dataset/Test/',12)

yawn_ted = yawn_detection_test('/content/drive/MyDrive/ColabNotebooks/AML_project/dataset/Test_y/',12)


Found 2880 images belonging to 2 classes.
Found 720 images belonging to 2 classes.
Found 1003 images belonging to 2 classes.
Found 249 images belonging to 2 classes.
Found 400 images belonging to 2 classes.
Found 196 images belonging to 2 classes.


In [10]:
def model():
  cnn_model = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3))) #1 if gray scale and 3 if colored
  out_cnn_model = cnn_model.output
  out_cnn_model = Flatten()(out_cnn_model)
  out_cnn_model = Dense(64, activation='relu')(out_cnn_model)
  out_cnn_model = Dropout(0.5)(out_cnn_model)
  out_cnn_model = Dense(2,activation= 'softmax')(out_cnn_model)
  f_model = Model(inputs=cnn_model.input, outputs= out_cnn_model)
  for i in cnn_model.layers:
    i.trainable = False
  return f_model




In [11]:
fmodel = model() 
fmodel.summary()

87910968/87910968 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 39, 39, 32)   864         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 39, 39, 32)  96          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                                                  
 activation (Activation)     

In [12]:
path_BestModel= '/content/drive/MyDrive/ColabNotebooks/AML_project/dataset/BestModel/'

model_cp = ModelCheckpoint(path_BestModel,monitor='val_loss',save_best_only=True,verbose=3)
model_es = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)
model_lr = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )
callbacks=[model_cp,model_es,model_lr]

In [13]:
fmodel.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

fmodel.fit_generator(eye_trd,steps_per_epoch=eye_trd.samples//12,
                   validation_data=eye_vd,
                   validation_steps=eye_vd.samples//12,
                   callbacks=callbacks,
                    epochs=5)

fmodel.fit_generator(yawn_trd,steps_per_epoch=yawn_trd.samples//12,
                   validation_data=yawn_vd,
                   validation_steps=yawn_vd.samples//12,
                   callbacks=callbacks,
                    epochs=5)

<ipython-input-13-31fb5062f16c>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fmodel.fit_generator(eye_trd,steps_per_epoch=eye_trd.samples//12,


Epoch 1/5
240/240 [==============================] - ETA: 0s - loss: 0.0440 - accuracy: 0.9847
Epoch 1: val_loss improved from inf to 0.29684, saving model to /content/drive/MyDrive/ColabNotebooks/AML_project/dataset/BestModel/


240/240 [==============================] - 474s 2s/step - loss: 0.0440 - accuracy: 0.9847 - val_loss: 0.2968 - val_accuracy: 0.9250 - lr: 0.0010
Epoch 2/5
240/240 [==============================] - ETA: 0s - loss: 0.0105 - accuracy: 0.9969
Epoch 2: val_loss improved from 0.29684 to 0.29586, saving model to /content/drive/MyDrive/ColabNotebooks/AML_project/dataset/BestModel/


240/240 [==============================] - 39s 163ms/step - loss: 0.0105 - accuracy: 0.9969 - val_loss: 0.2959 - val_accuracy: 0.9208 - lr: 0.0010
Epoch 3/5
240/240 [==============================] - ETA: 0s - loss: 0.0211 - accuracy: 0.9931
Epoch 3: val_loss did not improve from 0.29586
240/240 [==============================] - 15s 61ms/step - loss: 0.0211 - accuracy: 0.9931 - val_loss: 0.7958 - val_accuracy: 0.9153 - lr: 0.0010
Epoch 4/5
240/240 [==============================] - ETA: 0s - loss: 0.0206 - accuracy: 0.9948
Epoch 4: val_loss did not improve from 0.29586
240/240 [==============================] - 15s 63ms/step - loss: 0.0206 - accuracy: 0.9948 - val_loss: 0.6300 - val_accuracy: 0.9236 - lr: 0.0010
Epoch 5/5
240/240 [==============================] - ETA: 0s - loss: 0.0094 - accuracy: 0.9969
Epoch 5: val_loss did not improve from 0.29586

Epoch 5: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
240/240 [==============================] - 15s 63ms/step 

<ipython-input-13-31fb5062f16c>:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  fmodel.fit_generator(yawn_trd,steps_per_epoch=yawn_trd.samples//12,


Epoch 1/5
83/83 [==============================] - ETA: 0s - loss: 2.2279 - accuracy: 0.5358
Epoch 1: val_loss did not improve from 0.29586
83/83 [==============================] - 437s 5s/step - loss: 2.2279 - accuracy: 0.5358 - val_loss: 1.2785 - val_accuracy: 0.5208 - lr: 1.0000e-04
Epoch 2/5
83/83 [==============================] - ETA: 0s - loss: 0.9306 - accuracy: 0.5681
Epoch 2: val_loss did not improve from 0.29586
83/83 [==============================] - 11s 134ms/step - loss: 0.9306 - accuracy: 0.5681 - val_loss: 0.8728 - val_accuracy: 0.5125 - lr: 1.0000e-04
Epoch 3/5
83/83 [==============================] - ETA: 0s - loss: 0.7394 - accuracy: 0.5711
Epoch 3: val_loss did not improve from 0.29586
83/83 [==============================] - 11s 134ms/step - loss: 0.7394 - accuracy: 0.5711 - val_loss: 0.7858 - val_accuracy: 0.5250 - lr: 1.0000e-04
Epoch 4/5
83/83 [==============================] - ETA: 0s - loss: 0.7344 - accuracy: 0.5530
Epoch 4: val_loss did not improve from 0.2